## implementation of Bayesian Personalized Ranking for Implicit Feedback

In [1]:
import numpy as np
import data

In [2]:
train = data.train
test = data.test

In [3]:
class MatrixFactorization():
    
    def __init__(self, train, test, k, learning_rate, reg_param, epsilon):
        """
        param R : Rating Matrix
        param k : latent parameter
        param learning_rate : alpha on weight update
        param reg_param : regularization parameter
        """
        
        self._R = train
        self._test = test
        self._X = np.array(np.vectorize(lambda x: 0 if x==0 else 1)(train), dtype = np.float64) # create X matrix : implicit feedbacks (binary)
        self._num_users, self._num_items = train.shape
        self._k = k
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        self._epsilon = epsilon
        
        
    def fit(self):
        """
        training Matrix Factorization : update matrix latent weight and bias
        """
        
        # init latent features
        self._W = np.random.normal(scale = 1.0/self._k, size=(self._num_users, self._k))
        self._H = np.random.normal(scale = 1.0/self._k, size=(self._num_items, self._k))
        
        
        # train until cost converges
        self._training_process = []
        while converge == True :
            
            count += 1
                        
            train_cost, test_cost = self.cost()
            self._training_process.append((count, train_cost, test_cost))
            
            print("Iteration : %d, train_cost = %.4f, test_cost = %.4f" % (epoch+1, train_cost, test_cost))
    
    
    def sigmoid(self, x):
        """
        return sigmoid 
        """
        return 1 / (1 + np.exp(-x))
    
    
    def log_posterior(self):
        """
        compute log posterior of params
        """
        lp = 0
        non_zero_u = self._R.nonzero()[0] # 0 이 아닌 값의 index 반환
        
        for u in non_zero_u :
            temp_i = self._R[u].nonzero()
            for i in temp_i:
                for j in temp_i:
                    lp += np.log(self.sigmoid(self._W[u].dot((self._H[i]-self._H[j]).T)))
                    
        return lp - self._reg_param * (np.linalg.norm(self._W)+np.linalg.norm(self._H))
    
    
    def gradient_descent(self, u, i, j):
        """
        gradient descent function
        param u : user index
        param i : item index
        param j :
        param rating : rating of (u, i)
        """
        
        prediction = self.get_prediction(u,i)
        error = rating - prediction
        
        self._b_P[u] += self._learning_rate * (error - self._reg_param * self._b_P[u])
        self._b_Q[i] += self._learning_rate * (error - self._reg_param * self._b_Q[i])
        
        dp, dq, dy, j = self.gradient(error, u, i)
        self._P[u, :] += self._learning_rate * dp
        self._Q[i, :] += self._learning_rate * dq
        self._Y[j, :] += self._learning_rate * dy
        
        
    
    def get_prediction(self, u, i):
        """
        get predicted rating by user i on item j
        """
        
        return self._b + self._b_P[u] + self._b_Q[i] + self._Q[i, :].T.dot(self._P[u, :] + self._N[u] * self._Y[self._mask[u,:], :].sum(axis=0))

    
    def get_complete_matrix(self):
        """
        compute complete matrix
        """
        
        predictions = np.zeros([self._num_users, self._num_items])
        for u in range(self._num_users):
            for i in range(self._num_items):
                predictions[u, i] = self.get_prediction(u, i)
                
        predictions = np.array(predictions, dtype = np.float64)
        
        return predictions
    
    
    def print_results(self):
        """
        print fit results
        """

        print("Final R matrix:")
        print(self.get_complete_matrix())
        print("Final RMSE:")
        print(self._training_process[self._epochs-1][2])

In [4]:
np.random.seed(7)
np.seterr(all="warn")
    
factorizer = MatrixFactorization(train, test, k=40, learning_rate=0.001, reg_param=0.001, epochs=100, verbose=True)

# regression parameter 2개
factorizer.fit()
factorizer.print_results()

Iteration : 10, train_cost = 0.9487, test_cost = 0.9886
Iteration : 20, train_cost = 0.9196, test_cost = 0.9632
Iteration : 30, train_cost = 0.9069, test_cost = 0.9540
Iteration : 40, train_cost = 0.8935, test_cost = 0.9459
Iteration : 50, train_cost = 0.8784, test_cost = 0.9377
Iteration : 60, train_cost = 0.8636, test_cost = 0.9310
Iteration : 70, train_cost = 0.8480, test_cost = 0.9255
Iteration : 80, train_cost = 0.8311, test_cost = 0.9212
Iteration : 90, train_cost = 0.8126, test_cost = 0.9177
Iteration : 100, train_cost = 0.7928, test_cost = 0.9152
Final R matrix:
[[3.92795972 3.15884938 3.00837489 ... 3.10269225 3.50778327 3.33668068]
 [3.72351488 3.18190694 2.97444506 ... 3.32456955 3.63207592 3.54908656]
 [3.38509121 2.68161406 2.76964097 ... 2.9511531  3.19912569 3.16336248]
 ...
 [4.2410321  3.58504866 3.38590671 ... 3.57237379 3.79353462 3.73546977]
 [4.55871044 3.9119182  3.42007767 ... 3.69270164 3.94635209 3.84667653]
 [3.91530559 3.31412494 3.22606723 ... 3.05291738 3.2